In [1]:
#load "../Data/Data.fsx"
#r "nuget: Plotly.NET"
#r "nuget: Plotly.NET.Interactive"

open Data
open Plotly.NET

The below script needs to be able to find the current output cell; this is an easy method to get it.

Installed Packages Plotly.NET, 4.0.0 Plotly.NET.Interactive, 4.1.0

Loading extensions from `/Users/daniel/.nuget/packages/plotly.net.interactive/4.1.0/interactive-extensions/dotnet/Plotly.NET.Interactive.dll`

In [2]:
let allData = loadData()

let training, validation = allData |> splitData 0.7

In [3]:
let createModelChart (model: Model) =
    training
    |> Array.map (fun obs -> (float obs.Instant, model obs))
    |> Chart.Line



In [9]:
let cost (obs: Obs seq) (model: Model) =
    obs |> Seq.sumBy (fun obs -> (model obs - float obs.Cnt) ** 2.) |> sqrt
    
let trainingCost = cost training

let validationCost = cost validation

let overallCost = cost allData.Rows

In [10]:
let buildModel (theta0, theta1) (obs: Obs) = theta0 + theta1 * float obs.Instant

In [11]:
let evaluateModel model =
    validation |> Array.averageBy (fun obs -> abs (model obs - float obs.Cnt))
    

In [12]:
let updateThetas (alpha: float) (b, a) (obs: Obs) =
    let x = float obs.Instant
    let y = float obs.Cnt
    let b' = b - 2. * alpha * (b + a * x - y)
    let a' = a - 2. * alpha * x * (b + a * x - y)
    (b', a')
    
let batchUpdateThetas rate (theta0, theta1) (data: Obs seq) =
    let thetas = data |> Seq.map (updateThetas rate (theta0, theta1))

    let theta0' = thetas |> Seq.averageBy fst
    let theta1' = thetas |> Seq.averageBy snd
    (theta0', theta1')
    
let stochasticEstimation rate (theta0, theta1) =
    training |> Seq.fold (updateThetas rate) (theta0, theta1)
    
let batchEstimation rate iters (data : Obs seq)=
    let rec search (theta0, theta1) i =
        if i = 0 then (theta0, theta1)
        else
            search (batchUpdateThetas rate (theta0, theta1) data) (i-1)
    
    search (0.,0.) iters
    


In [13]:
let rate =
    [ for r in 1..30 -> (pown 0.1 r), stochasticEstimation (pown 0.1 r) (0., 0.) |> buildModel |> trainingCost ]
    |> Seq.filter (fun (rate, error) -> error > 0)
    |> Seq.minBy snd
    |> fst

In [17]:
let baselineModel =
    fun (_: Obs) -> allData.Rows |> Seq.averageBy (fun x -> float x.Cnt)

In [18]:
let thetas1 = stochasticEstimation rate (0., 0.)
let model1 = thetas1 |> buildModel

In [19]:
let thetas2 = batchEstimation rate 2000 training
let model2 = thetas2 |> buildModel

In [20]:
evaluateModel baselineModel |> printfn "Naive model average error: %f"
thetas1 |> printfn "Model 1 thetas: %A"
evaluateModel model1 |> printfn "Model 1 average error: %f"
thetas2 |> printfn "Model 2 thetas: %A"
evaluateModel model2 |> printfn "Model 2 average error: %f"

Naive model average error: 1619.742275
Model 1 thetas: (0.08445985761, 9.643593597)
Model 1 average error: 1539.519699
Model 2 thetas: (0.2753406868, 10.57574839)
Model 2 average error: 1426.257911


In [22]:
let chart =
    allData.Rows
    |> Seq.map (fun obs -> (float obs.Instant, float obs.Cnt))
    |> Chart.Point
    
    

let model1Chart = createModelChart model1

let model2Chart = createModelChart model2

[ chart; baselineChart; model1Chart; model2Chart ]
|> Chart.combine



input.fsx (10,10)-(10,23) typecheck error Nie zdefiniowano wartości lub konstruktora „baselineChart”. Możliwe, że chcesz wykonać jedną z następujących czynności:
   baselineModel



Error: compilation error